In [2]:
!pip install transformers trl peft accelerate Dataset bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 kB 6.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.30
    Uninstalling SQLAlchemy-2.0.30:
      Successfully uninstalled SQLAlchemy-2.0.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 

In [3]:
from peft import (
    LoraConfig,
    PeftModel,
    TaskType,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from sklearn.model_selection import train_test_split
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)
from datasets import Dataset, load_dataset
from trl import DataCollatorForCompletionOnlyLM, SFTConfig, SFTTrainer
import torch

In [4]:
PAD_TOKEN = "<|pad|>"
MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.add_special_tokens({"pad_token": PAD_TOKEN})
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto",
)
model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=8)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Embedding(32016, 3072, padding_idx=32000)

In [5]:
input = "what is machine learninig"
input_token = tokenizer.encode(input, return_tensors="pt", truncation=True, padding=True, max_length=128)
output_token = model.generate(input_token, max_length=128)
output_text = tokenizer.decode(output_token[0], skip_special_tokens=True)
print(output_text)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1935: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


what is machine learninig?

# Answer
Machine learning is a subset of artificial intelligence (AI) that focuses on building systems that can learn from and make decisions based on data. It involves the development of algorithms that can learn from and make predictions or decisions based on input data, without being explicitly programmed to perform the task. Machine learning algorithms use statistical methods to enable computers to improve at a task with experience. This field has applications across various industries, including finance, healthcare, transportation, and more, where it can be used for tasks such as fraud detection, disease diagnosis,


In [41]:
lora_config = LoraConfig(
    r=32,
    lora_alpha = 32,
    lora_dropout = 0.05,
    target_modules=[
        "self_attn.q_proj",
        "self_attn.k_proj",
        "self_attn.v_proj",
        "self_attn.o_proj",
        "mlp.gate_proj",
        "mlp.up_proj",
        "mlp.down_proj",
    ],
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): PeftModelForCausalLM(
          (base_model): LoraModel(
            (model): PeftModelForCausalLM(
              (base_model): LoraModel(
                (model): PeftModelForCausalLM(
                  (base_model): LoraModel(
                    (model): PeftModelForCausalLM(
                      (base_model): LoraModel(
                        (model): Phi3ForCausalLM(
                          (model): Phi3Model(
                            (embed_tokens): Embedding(32016, 3072, padding_idx=32000)
                            (embed_dropout): Dropout(p=0.0, inplace=False)
                            (layers): ModuleList(
                              (0-31): 32 x Phi3DecoderLayer(
                                (self_attn): Phi3SdpaAttention(
                                  (o_proj): lora.Linear4bit(
                                    (base_layer)

In [44]:
model.print_trainable_parameters()

trainable params: 17,825,792 || all params: 3,838,610,432 || trainable%: 0.4644


In [43]:
df = load_dataset("MakTek/Customer_support_faqs_dataset",split="train")
df

Dataset({
    features: ['question', 'answer'],
    num_rows: 200
})

In [23]:
df.to_pandas()

,question,answer
0,How can I create an account?,"To create an account, click on the 'Sign Up' b..."
1,What payment methods do you accept?,"We accept major credit cards, debit cards, and..."
2,How can I track my order?,You can track your order by logging into your ...
3,What is your return policy?,Our return policy allows you to return product...
4,Can I cancel my order?,You can cancel your order if it has not been s...
...,...,...
195,Do you offer a satisfaction guarantee?,"Yes, we offer a satisfaction guarantee on our ..."
196,How can I apply for a job at your company?,"To apply for a job at our company, visit our C..."
197,What is the warranty on your products?,The warranty on our products varies by item. P...
198,Can I request a refund if the price drops afte...,If the price of a product drops within 7 days ...


In [28]:
def tokenize_function(examples):
    return tokenizer(examples['question'], examples['answer'], truncation=True)


# Tokenize the dataset
tokenized_dataset = df.map(tokenize_function, batched=True)

tokenized_dataset

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 200
})

In [31]:
tokenized_dataset.to_pandas()

,question,answer,input_ids,attention_mask
0,How can I create an account?,"To create an account, click on the 'Sign Up' b...","[1128, 508, 306, 1653, 385, 3633, 29973, 1763,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,What payment methods do you accept?,"We accept major credit cards, debit cards, and...","[1724, 19179, 3519, 437, 366, 3544, 29973, 133...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,How can I track my order?,You can track your order by logging into your ...,"[1128, 508, 306, 5702, 590, 1797, 29973, 887, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,What is your return policy?,Our return policy allows you to return product...,"[1724, 338, 596, 736, 8898, 29973, 8680, 736, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Can I cancel my order?,You can cancel your order if it has not been s...,"[1815, 306, 12611, 590, 1797, 29973, 887, 508,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
195,Do you offer a satisfaction guarantee?,"Yes, we offer a satisfaction guarantee on our ...","[1938, 366, 5957, 263, 26470, 18818, 29973, 38...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
196,How can I apply for a job at your company?,"To apply for a job at our company, visit our C...","[1128, 508, 306, 3394, 363, 263, 4982, 472, 59...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
197,What is the warranty on your products?,The warranty on our products varies by item. P...,"[1724, 338, 278, 1370, 21867, 29891, 373, 596,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
198,Can I request a refund if the price drops afte...,If the price of a product drops within 7 days ...,"[1815, 306, 2009, 263, 2143, 870, 565, 278, 86...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [52]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [46]:
OUTPUT_DIR = "experiments"
sft_config = SFTConfig(
    output_dir=OUTPUT_DIR,
    dataset_text_field="text",
    max_seq_length=512,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    eval_strategy="steps",
    eval_steps=0.2,
    save_steps=0.2,
    logging_steps=10,
    learning_rate=1e-4,
    fp16=True,  # or bf16=True,
    save_strategy="steps",
    warmup_ratio=0.1,
    save_total_limit=2,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    save_safetensors=True,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False,  # No need to add additional separator token
    }
)

In [55]:
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [56]:
trainer.train()

Step,Training Loss,Validation Loss
8,No log,0.708395
16,0.857700,0.530556
24,0.579500,0.437350
32,0.415300,0.381245
40,0.334200,0.351740


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=40, training_loss=0.5466714859008789, metrics={'train_runtime': 166.698, 'train_samples_per_second': 1.92, 'train_steps_per_second': 0.24, 'total_flos': 341111463936000.0, 'train_loss': 0.5466714859008789, 'epoch': 2.0})

In [58]:
trainer.evaluate()

{'eval_loss': 0.35174018144607544,
 'eval_runtime': 6.4966,
 'eval_samples_per_second': 6.157,
 'eval_steps_per_second': 3.079,
 'epoch': 2.0}

In [89]:
def create_prompt(question):
    # Define a template with clear instructions for the model
    prompt = (
        "You are a helpful customer support assistant. "
        "Answer the question clearly and concisely from the provided dataset."
        "Do not repeat the question in your answer.\n\n"
        "Question: " + question + "\n"
        "Answer:"
    )
    return prompt

def generate_response(question):
    
#     prompts = create_prompt(question)
    inputs = tokenizer.encode(question, return_tensors="pt", padding=True, truncation=True)
    
    # Generate the outputs for the question
    outputs = model.generate(
        inputs,
        max_length=50,
        num_beams=4,
        early_stopping=True
    )
    
    # Decode the outputs, ensuring to skip special tokens
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

# Example question
question = "What payment methods do you accept?	"
answer = generate_response(question)
print("Question:"+question)
print("Answer:"+answer)

Question:What payment methods do you accept?	
Answer:What payment methods do you accept?	We accept major credit cards, such as Visa, Mastercard, and American Express. You can also make payments using PayPal. Please contact our customer support team for more information.

3.


In [93]:

model.save_pretrained("new_model-7B-mini-4k-instruct")  # Save the model
tokenizer.save_pretrained("new_model-7B-mini-4k-instruct")

('new_model-7B-mini-4k-instruct/tokenizer_config.json',
 'new_model-7B-mini-4k-instruct/special_tokens_map.json',
 'new_model-7B-mini-4k-instruct/tokenizer.model',
 'new_model-7B-mini-4k-instruct/added_tokens.json',
 'new_model-7B-mini-4k-instruct/tokenizer.json')